# Segmenting and Clustering Neighborhoods in Toronto

In [21]:
# using requests load given web-page into object page 
import requests
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page

<Response [200]>

In [22]:
import pandas as pd

In [23]:
!pip install beautifulsoup4

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [24]:
!conda install -c anaconda beautifulsoup4

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2s             |       h7b6447c_0         3.1 MB  anaconda
    certifi-2018.8.24          |           py35_1         139 KB  anaconda
    ca-certificates-2019.5.15  |                0         133 KB  anaconda
    beautifulsoup4-4.6.3       |           py35_0         140 KB  anaconda
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0      anaconda
    ca-certificates: 2019.1.23-0          --> 2019.5.15-0       anaconda
    certifi:         2018.8.24-py35_1     --> 2018.8.24-py35_1  anaconda
    openssl:   

In [25]:
#here we parse page content
#page.content
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [26]:
#print(soup.prettify())

In [27]:
#Let's create dataframe for our data. It consists of 3 columns  
CACodeDf = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

In [32]:
#And finaly, let's load data from our page into dataframe  CACodeDf
i = 0
for tr in soup.find_all('tr', text=True)[1:]:
    tds = tr.find_all('td')  
    my_str = tds[2].text.replace('\n', '')
    CACodeDf.loc[i] = [tds[0].text]+ [tds[1].text] + [my_str]
    i = i+1
    #print(tds[0].text)
print      ("done")

done


In [33]:
CACodeDf.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [34]:
CACodeDf.shape

(288, 3)

In [35]:
#Now , since we have dataframe filled with all necessary data, we should clear it from inappropriate rows
CACodeDf = CACodeDf[CACodeDf.Borough != "Not assigned"]
CACodeDf.head(10) 

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [10]:
CACodeDf.shape

(211, 3)

In [36]:
CACodeDf.loc[CACodeDf['Neighborhood'] == "Not assigned", ['Neighborhood']] = CACodeDf['Borough']
CACodeDf.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [37]:
CACodeDf.shape

(211, 3)

In [38]:
df = CACodeDf[['PostalCode','Borough']]
#df.drop_duplicates()
df=df.drop_duplicates(['PostalCode','Borough'])[['PostalCode','Borough']]
df['Neighborhood'] = df.apply(lambda row: "", axis=1)
df.head(10)
#df.shape

,PostalCode,Borough,Neighborhood
2,M3A,North York,
3,M4A,North York,
4,M5A,Downtown Toronto,
6,M6A,North York,
8,M7A,Queen's Park,
10,M9A,Etobicoke,
11,M1B,Scarborough,
14,M3B,North York,
15,M4B,East York,
17,M5B,Downtown Toronto,


In [39]:
for index, row in df.iterrows():
    my_str = ''
    for index2, row2 in CACodeDf.iterrows():
        if row2['PostalCode'] == row['PostalCode'] and row2['Borough'] == row['Borough']:
            my_str = row2['Neighborhood']+","+my_str
    row['Neighborhood'] = my_str[:-1]
df.head(10)     

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"
6,M6A,North York,"Lawrence Manor,Lawrence Heights"
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Malvern,Rouge"
14,M3B,North York,Don Mills North
15,M4B,East York,"Parkview Hill,Woodbine Gardens"
17,M5B,Downtown Toronto,"Garden District,Ryerson"


In [40]:
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data

--2019-06-30 13:26:31--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-30 13:26:31--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-30 13:26:32--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-30 

In [41]:
df_coord = pd.read_csv('Geospatial_Coordinates.csv')
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
df['Latitude'] = df.apply(lambda row: "", axis=1)
df['Longitude'] = df.apply(lambda row: "", axis=1)
df.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,,
3,M4A,North York,Victoria Village,,
4,M5A,Downtown Toronto,"Regent Park,Harbourfront",,
6,M6A,North York,"Lawrence Manor,Lawrence Heights",,
8,M7A,Queen's Park,Queen's Park,,


In [50]:
for index, row in df.iterrows():
    for index2, row2 in df_coord.iterrows():
        if row2['Postal Code'] == row['PostalCode'] :            
            row['Latitude'] = row2['Latitude']
            row['Longitude'] = row2['Longitude']
df.reset_index(inplace=True)
df.reset_index(inplace=True)
df.head(10)    

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.7533,-79.3297
1,3,M4A,North York,Victoria Village,43.7259,-79.3156
2,4,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.6543,-79.3606
3,6,M6A,North York,"Lawrence Manor,Lawrence Heights",43.7185,-79.4648
4,8,M7A,Queen's Park,Queen's Park,43.6623,-79.3895
5,10,M9A,Etobicoke,Islington Avenue,43.6679,-79.5322
6,11,M1B,Scarborough,"Malvern,Rouge",43.8067,-79.1944
7,14,M3B,North York,Don Mills North,43.7459,-79.3522
8,15,M4B,East York,"Parkview Hill,Woodbine Gardens",43.7064,-79.3099
9,17,M5B,Downtown Toronto,"Garden District,Ryerson",43.6572,-79.3789
